In [9]:
def complex_fun(row):
    # some complex thing
    return row['a'] + 1

In [2]:
import pandas as pd

In [12]:
df = pd.DataFrame({'a' : [1, 2, 3, 4], 'b' : [2, 34 ,56 ,12]})

In [47]:
df

,a,b,c
0,1,2,2
1,2,34,3
2,3,56,4
3,4,12,5
